In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import gdown
import os
import zipfile

In [ ]:
#Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

#Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  # Replace with actual file name

#Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  # Name of the downloaded file

#Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

#Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F
From (redirected): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F&confirm=t&uuid=1907dffb-f56a-460e-a307-831df4086600
To: /content/disease.zip
100%|██████████| 3.85G/3.85G [00:56<00:00, 68.1MB/s]


In [ ]:
#Path to main train folder that has many class folders
data_dir = "/content/extracted_file/Retinal Fundus Images - Copy/train"

#Specify the exact classes you want to include
target_classes = ['Cataract', 'Normal_Fundus']

#Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

#Create generators with only selected classes
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

#Model definition
def build_ImprovedCNN():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        GlobalAveragePooling2D(),

        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(2, activation='softmax')  #Only 2 classes: AMD and Normal_Fundus
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

#Callbacks
callbacks = [
    ModelCheckpoint("best_model_rgb.keras", save_best_only=True, monitor='val_accuracy', mode='max')
]

#Build and train model
model = build_ImprovedCNN()
#model.summary()

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)

#Evaluate
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


Found 18046 images belonging to 2 classes.
Found 2004 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 134s 199ms/step - accuracy: 0.8890 - loss: 0.3131 - val_accuracy: 0.3413 - val_loss: 7.1328
Epoch 2/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 84s 148ms/step - accuracy: 0.9553 - loss: 0.1143 - val_accuracy: 0.9810 - val_loss: 0.0563
Epoch 3/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 83s 147ms/step - accuracy: 0.9686 - loss: 0.0810 - val_accuracy: 0.9711 - val_loss: 0.0638
Epoch 4/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 83s 148ms/step - accuracy: 0.9788 - loss: 0.0624 - val_accuracy: 0.7191 - val_loss: 1.0254
Epoch 5/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 83s 146ms/step - accuracy: 0.9802 - loss: 0.0528 - val_accuracy: 0.9930 - val_loss: 0.0232
Epoch 6/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 147s 155ms/step - accuracy: 0.9842 - loss: 0.0491 - val_accuracy: 0.9616 - val_loss: 0.0830
Epoch 7/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 83s 147ms/step - accuracy: 0.9839 - loss: 0.0472 - val_accuracy: 0.9915 - val_loss: 0.0218
Epoch 8/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 82s 146ms/step - accuracy: 0.9850 - loss:

In [4]:
model.save('/content/Cataract.keras')